In [13]:
# 주피터 노트북 가상환경 커널 연결
# https://taehooh.tistory.com/entry/Python-%EC%95%84%EB%82%98%EC%BD%98%EB%8B%A4-%EA%B0%80%EC%83%81%ED%99%98%EA%B2%BD-%EA%B5%AC%EC%84%B1-%EB%B0%8F-%EC%A3%BC%ED%94%BC%ED%84%B0-%EB%85%B8%ED%8A%B8%EB%B6%81-%EC%BB%A4%EB%84%90-%EC%97%B0%EA%B2%B0
# python -m ipykernel install --user --name 가상환경이름 --display-name "표시할 커널이름"
# python -m ipykernel install --user --name asac --display-name "asac"

from selenium import webdriver
import time
from bs4 import BeautifulSoup
import re
import pandas as pd

url = r'https://www.wadiz.kr/web/wreward/category/294?endYn=Y&order=recent'


# 최신순, 종료된 URL : https://www.wadiz.kr/web/wreward/category/316?endYn=Y&order=recent
# 전체            : https://www.wadiz.kr/web/wreward/main?endYn=Y&order=recent
# BSET 펀딩       : https://www.wadiz.kr/web/wreward/category/categorybanner?endYn=Y&order=recent

# 테크-가전       : 287
# 패션-잡화       : 288
# 홈-리빙         : 310
# 뷰티            : 311
# 푸드            : 289
# 출판            : 293
# 클래스-컨설팅   : 316
# 레저-아웃도어   : 296
# 스포츠-모빌리티 : 297
# 컬쳐-아티스트   : 294
# 캐릭터-굿즈     : 290
# 반려동물        : 308
# 베이비-키즈     : 309
# 게임-취미       : 292
# 여행-숙박       : 317
# 기부-캠페인     : 295
# 후원            : 312
# 모임            : 313




# 크롬 브라우저를 열기 위해 Chrome 드라이버 사용
driver = webdriver.Chrome(r"C:\chrome_driver\chromedriver.exe")

# Chrome 버전 117.0.5938.89(공식 빌드) (64비트)
# 크롬드라이버 다운로드 https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/117.0.5938.88/win64/chromedriver-win64.zip

# 해당 URL에 접속
driver.get(url)

# 현재 높이 저장
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")   # 스크롤을 끝까지 내림
    time.sleep(2)                                                              # 로딩을 기다리기 위해 2초간 대기
    
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:                                             # 스크롤이 더 이상 이동하지 않으면 반복문을 종료
        break
    last_height = new_height                                                  # 현재 높이 저장

html = driver.page_source                                                     # 페이지의 HTML 코드를 가져옴

# 브라우저 종료
# driver.quit()

soup = BeautifulSoup(html, 'html.parser')

titles = soup.find_all('div', {'class': 'PreorderMainCard_title__XlTJk'})
percent = soup.find_all('p', class_='PreorderMainCard_participants__JDp58')
tot_amount = soup.find_all('p', class_='PreorderMainCard_amount__3UOXm')
company = soup.find_all("span", {"class": "PreorderMainCard_makerName__2Ufj1"})

data = []

for i in range(len(titles)):
    temp = {
        'title': titles[i].text.strip(),
        'percent': re.sub('[^0-9]', '', percent[i].text.strip()),
        'total amount': re.sub('[^0-9]', '', tot_amount[i].text.strip()),
        'company': company[i].text.strip()
    }
    data.append(temp)

df = pd.DataFrame(data)


# 데이터프레임 내 허용되지 않는 문자 삭제
for column in df.columns:
    df[column] = df[column].apply(lambda x: ''.join(e for e in x if e.isalnum() or e.isspace()))


df.to_excel('output_컬쳐아티스트.xlsx', index=False)
print("complete!!")

C:\Users\heniz\AppData\Local\Temp\ipykernel_4112\961948561.py:42: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\chrome_driver\chromedriver.exe")


complete!!
